### 정규 표현식 사용 
- 문자열의 패턴 매칭이 필요한 상황은 자주 발생 
    - 로그 파일의 식별자 찾기
    - 사용자의 입력에서 키워드 체크
- 정규 표현식은 문자열을 사용해 검색 패턴을 정의 
- 파이썬의 re 패키지는 Perl과 유사한 정규식 연산을 제공
- re 모듈은 \ 를 사용해 매칭에 사용되는 특수 문자 기술 
- 이스케이프 시퀀스와의 혼동을 방지하기 위해 정규 표현식을 사용할 때 원시 문자열을 사용하는게 좋음
- 원시 문자열이란 첫 번째 따옴표 앞에 'r'이 붙는다.

### 검색
- 텍스트로 된 이메일의 참조 리스트가 있는데 이 리스트에 누가 있는지 알고 싶다고 가정 

In [1]:
cc_list = '''
Ezra Koenig <ekoenig@vpwk.com>,
Rostam Batmanglij <rostam@vpwk.com>,
Chris Tomson <comson@vpwk.com>,
Bobbi Baio <bbaio@vpwk.com>
'''

In [2]:
# 리스트에 특정 이름이 있는지 알고 싶다면 in 시퀀스 멤버십 구문을 사용
'Rostam' in cc_list

True

In [3]:
# 일치하는 항목이 있을 경우에만 re.Match 객체를 반환해주는 re.search 함수 사용
import re

re.search(r'Rostam', cc_list)

<_sre.SRE_Match object; span=(33, 39), match='Rostam'>

In [4]:
# 특정 항목이 존재하는지 확인하기 위한 조건으로 사용할 수 있음 
if re.search(r'Rostam', cc_list):
    print('Found Rostam')

Found Rostam


### 캐릭터 세트 
- 정규 표현식을 사용하면 문자 그룹을 사용할 수 있으며 1개라도 나타날 수 있음 
- 이를 캐릭터 세트 (character set)라 한다. 
- 선택적으로 일치할 수 있는 문자는 정규 표현식의 정의에서 대괄호로 묶어서 나타냄 

In [7]:
# B 또는 R로 시작하면서 obb가 이어지고 i 혹은 y로 |끝나는 이름을 찾는 것
re.search(r'[RB]obb[i,y]', cc_list)

<_sre.SRE_Match object; span=(102, 107), match='Bobbi'>

In [9]:
# 콤마로 구분된 개별 문자를 입력하거나 범위를 사용할 수 도 있음 
# A-Z는 대문자 모두를 포함, 0-9는 0~9까지의 모든 숫자 포함 
re.search(r'Chr[a-z][a-z]', cc_list)

<_sre.SRE_Match object; span=(70, 75), match='Chris'>

In [10]:
# 정규 표현식에서 어떤 항목 뒤에 있는 +는 해당 하목이 한 번 이상 나타나는 것을 찾아줌
re.search(r'[A-Za-z]+', cc_list)

<_sre.SRE_Match object; span=(1, 5), match='Ezra'>

In [11]:
# 대괄호 안의 숫자는 문자의 개수가 정확히 일치하는 것을 찾음 
re.search(r'[A-Za-z]{6}', cc_list)

<_sre.SRE_Match object; span=(6, 12), match='Koenig'>

In [12]:
# '.' 문자는 어떤 문자와도 매칭할 수 있는 와일드카드
# 실제 '.' 문자와 일치하는 것을 찾으려면 백슬래시를 사용해 이스케이프 처리를 해야함 
re.search(r'[A-Za-z]+@[a-z]+\.[a-z]+', cc_list)

<_sre.SRE_Match object; span=(14, 30), match='ekoenig@vpwk.com'>

### 캐릭터 클래스
- 일반적으로 사용되는 것은 \w로 이는 [a-zA-Z0-9_]에 해당 
- \d는 [0-9]dp goekd 
- 다중 문자의 일치는 +를 사용해 처리

In [13]:
re.search(r'\w+', cc_list)

<_sre.SRE_Match object; span=(1, 5), match='Ezra'>

In [14]:
# 기본적인 이메일 처리 
re.search(r'\w+@\w+\.\w+', cc_list)

<_sre.SRE_Match object; span=(14, 30), match='ekoenig@vpwk.com'>

### 그룹
- 괄호를 사용해 일치한 대상에서 그룹을 정의 가능 
- 그룹은 일치한 객체에서 접근할 수 있는데, 나타난 순서대로 번호가 매겨짐
- 0 그룹은 전체 일치를 의미

In [15]:
re.search(r'(\w+)\@(\w+)\.(\w+)', cc_list)

<_sre.SRE_Match object; span=(14, 30), match='ekoenig@vpwk.com'>

In [16]:
matched = re.search(r'(\w+)\@(\w+)\.(\w+)', cc_list)
matched.group(0)

'ekoenig@vpwk.com'

In [18]:
print(matched.group(1))
print(matched.group(2))
print(matched.group(3))


ekoenig
vpwk
com


### 네임드 그룹
- 그룹 정의에 ?P<NAME>을 추가해 해당 그룹에 이름을 부여할 수 있다.
- 이후부터는 번호 대신에 이름을 통해 그룹에 접근 가능

In [22]:
matched = re.search(r'(?P<name>\w+)\@(?P<SLD>\w+)\.(?P<TLD>\w+)', cc_list)
print(matched.group('name'))
print(matched.group('SLD'))
print(matched.group('TLD'))
print(f'''
name : {matched.group('name')},
Secondary Level Domain : {matched.group('SLD')},
Top Level Domain : {matched.group('TLD')}
''')

ekoenig
vpwk
com

name : ekoenig,
Secondary Level Domain : vpwk,
Top Level Domain : com



### 모두 찾기 
- findall을 사용하면 일치하는 모든 항목을 문자열 리스트로 받을 수 있음

In [26]:
matched = re.findall(r'\w+\@\w+\.\w+', cc_list)
print(matched) 

matched = re.findall(r'(\w+)\@(\w+)\.(\w+)', cc_list)
print(matched)

names = [x[0] for x in matched] 
print(names) 

['ekoenig@vpwk.com', 'rostam@vpwk.com', 'comson@vpwk.com', 'bbaio@vpwk.com']
[('ekoenig', 'vpwk', 'com'), ('rostam', 'vpwk', 'com'), ('comson', 'vpwk', 'com'), ('bbaio', 'vpwk', 'com')]
['ekoenig', 'rostam', 'comson', 'bbaio']


### 찾기 반복자
- finditer 메서드를 사용해 반복자 (iterator) 객체 생성
- 반복자 객체는 텍스트를 처리하다가 일치하는 항목을 찾으면 멈춤 
- 현재 찾은 내용을 next 함수를 통해 반환하고 다음 일치하는 항목을 찾을 때까지 이어서 처리 
- 모든 입력을 한 번에 처리하느라 리소스를 쏟지 않고도 일치하는 항목을 1개씩 찾을 수 있음 

In [31]:
matched = re.finditer(r'\w+\@\w+\.\w+', cc_list)
print(matched)
print(next(matched))
print(next(matched))
print(next(matched))

<_sre.SRE_Match object; span=(14, 30), match='ekoenig@vpwk.com'>
<_sre.SRE_Match object; span=(52, 67), match='rostam@vpwk.com'>
<_sre.SRE_Match object; span=(84, 99), match='comson@vpwk.com'>


In [36]:
matched = re.finditer(r'(?P<name>\w+)\@(?P<SLD>\w+)\.(?P<TLD>\w+)', cc_list)
for m in matched : 
    print(m.groupdict())

{'name': 'ekoenig', 'SLD': 'vpwk', 'TLD': 'com'}
{'name': 'rostam', 'SLD': 'vpwk', 'TLD': 'com'}
{'name': 'comson', 'SLD': 'vpwk', 'TLD': 'com'}
{'name': 'bbaio', 'SLD': 'vpwk', 'TLD': 'com'}


### 치환 
- 정규 표현식을 사용하면 검색과 매칭 이외에도 문자열의 일부 또는 전체를 치환 가능 

In [39]:
print(re.sub("\d", "#", "The passcode you entered was 09876"))

The passcode you entered was #####


In [41]:
users = re.sub("(?P<name>\w+)\@(?P<SLD>\w+)\.(?P<TLD>\w+)", 
                "\g<TLD>.\g<SLD>.\g<name>", cc_list)
print(users)


Ezra Koenig <com.vpwk.ekoenig>,
Rostam Batmanglij <com.vpwk.rostam>,
Chris Tomson <com.vpwk.comson>,
Bobbi Baio <com.vpwk.bbaio>



### 컴파일러
- 동일한 일치 항목이 많이 발생하는 경우에는 정규 표현식을 하나의 객체로 컴파일해 성능상의 이점 얻음
- 이 객체는 재컴파일 없이 재사용 가능

In [43]:
regex = re.compile(r'\w+\@\w+\.\w+')
regex.search(cc_list)

<_sre.SRE_Match object; span=(14, 30), match='ekoenig@vpwk.com'>

### 지연 평가
- 지연 평가 (lazy evaluation)는 대량의 데이터를 다룰 때 사용 
- 결과를 사용하기 전에 모든 데이터를 처리하지 않고자 하는 것 

### 제네레이터 
- range 객체와 유사한 방식으로 generator를 사용 가능 
- 요청에 따라 데이터 통째로 연산을 수행
- 제네레이터는 호출 사이에서 자신의 상태를 일시 정지 시킴 
- 이는 출력을 계산하는데 필요한 변수를 저장할 수 있고 변수들은 호출할 때마다 접근이 이뤄지는 것을 의미
- 제네레이터 함수를 작성하기 위해서는 return 구문보다 yield 키워드를 사용 
- 매번 제네레이터가 호출되면 yield에서 지정한 값을 반환하고 호출이 있을 때까지 자신의 상태를 정지시킴

In [1]:
def count() : 
    n = 0
    while True : 
        n += 1
        yield n

counter = count() 
print(counter)
print(next(counter))
print(next(counter))
print(next(counter))

<generator object count at 0x000001D911D506D0>
1
2
3


In [5]:
# 피보나치 제네레이터 구현 
def fib() :
    first = 0 
    last = 1 
    while True : 
        first, last = last, first + last 
        yield first 

f = fib() 
print(next(f))
print(next(f))
print(next(f))
print(next(f))

for x in f : 
    print(x)
    if x > 12 : 
        break

1
1
2
3
5
8
13


### 제네레이터 내포
- 내포를 사용해 한 줄 제네레이터를 만들 수 있음 
- 리스트 내포와 유사한 문법으로 만들지만 대괄호 대신 괄호를 사용

In [6]:
list_o_nums = [x for x in range(100)]
gen_o_nuks = (x for x in range(100))
print(list_o_nums)
print(gen_o_nuks)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
<generator object <genexpr> at 0x000001D911D50B48>


In [8]:
# 객체의 크기를 바이트로 반환해주는 sys.getsizeof 메소드 
import sys
print(sys.getsizeof(list_o_nums))
print(sys.getsizeof(gen_o_nuks))

912
88
